In [1]:
import tensorflow as tf

In [2]:
inception = tf.keras.applications.InceptionV3

In [3]:
dir(inception)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_keras_api_names',
 '_keras_api_names_v1']

In [4]:
model = inception()

96116736/96112376 [==============================] - 68s 1us/step


In [5]:
# connected_to 라는게 생겼다.

model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [6]:
vgg = tf.keras.applications.VGG16()

In [7]:
# inception과 차이점: connected_to 가 없다.

vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
mobile = tf.keras.applications.MobileNetV2()

In [9]:
# relu layer가 있다.

mobile.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

> InceptionV3: 
- summary에 Connected to 가 있다.
- Activation 레이어가 있다. (Relu Layer)  

>VGG16:    
- summary에 Connected to 가 없다.
    - Multi input, output이 없기 때문이다.

>MobileNetV2: 
- summary에 Connected to 가 있다.
- Activation 레이어가 있다. (Relu Layer)  
    - Activation은 BatchNormalization의 다음에 와야한다.
    - Keras 구현상의 이슈때문이다.
        - Activation 하기 전에 BatchNormalization을 적용해야하는데, Activation을 layer의 옵션으로 주면 BatchNormalization을 할 수가 없다. 그래서 Activation Layer를 따로 만들었다.  
    - 이론으로 배우는 모델 구조와 summary는 조금 다르다.
    - Layer에 activation을 옵션으로 주면 기능은 똑같지만 summary에는 안 보인다. 
        - 구현적인 관점에서는 장점이지만 이론적 관점에서는 단점일 수 있다.

---

In [10]:
from tensorflow.keras.layers import Conv2D

model = tf.keras.Sequential([Conv2D(32, (3,3), input_shape=(28,28,1)),
                            Conv2D(64, (3,3)),
                            Conv2D(128, (3,3))])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_94 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 22, 22, 128)       73856     
Total params: 92,672
Trainable params: 92,672
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 첫번째 레이어의 parmaeter 수
# kernel size = 3x3
# bias 개수 = 1
# filter 개수 = 32

(3*3+1)*32

320

In [17]:
import sympy

In [38]:
kh, kw, bias, fs, dims = sympy.symbols("H_kerenl W_kernel bias N_filter, Dim_input")

In [39]:
t = (kh * kw * dims + bias)* fs
t

N_filter*(Dim_input*H_kerenl*W_kernel + bias)

In [34]:
paramCount = sympy.lambdify((kh, kw, dims, bias, fs),t)

In [35]:
# 두번째 레이어
# kernel size = 3x3
# input Dimension = 32
# bias 개수 = 1
# filter 개수 = 64

paramCount(3,3,32,1,64)

18496

In [37]:
# 세번째 레이어
# kernel size = 3x3
# input Dimension = 64
# bias 개수 = 1
# filter 개수 = 128

paramCount(3,3,64,1,128)

73856